In [18]:
#import data manipulation libraries
import numpy as np
import pandas as pd

#import data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#import filter warning libraries
import warnings
warnings.filterwarnings('ignore')

#import logging libraries
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s',
                    filename='model.log',
                    filemode='w')
#importing scikit learn libraries
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix


In [20]:
#importing dataset using pandas function
url='https://raw.githubusercontent.com/preet-pandey/Bank_Classification_Model/refs/heads/main/bank.csv'
df=pd.read_csv(url,sep=';')
df.sample(frac=1)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
3692,36,blue-collar,divorced,primary,no,117,yes,no,unknown,2,jun,123,4,-1,0,unknown,no
572,53,management,married,tertiary,no,1319,yes,no,cellular,16,apr,427,1,150,1,failure,no
4226,37,blue-collar,married,secondary,no,395,yes,no,unknown,6,may,219,2,-1,0,unknown,no
2871,32,management,married,tertiary,no,2431,yes,no,cellular,14,may,646,5,-1,0,unknown,yes
3481,40,unemployed,married,secondary,no,0,yes,no,cellular,19,nov,332,1,183,1,failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4107,50,blue-collar,married,primary,no,2142,yes,no,cellular,29,jan,309,2,224,1,other,no
2350,34,technician,married,secondary,no,0,yes,no,cellular,29,aug,116,4,-1,0,unknown,no
2408,46,blue-collar,married,secondary,no,0,yes,no,cellular,8,may,264,2,-1,0,unknown,no
2828,38,technician,married,secondary,no,1960,yes,no,cellular,21,aug,161,2,-1,0,unknown,no


In [22]:
#dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [38]:
#step 1 : segregate columns categorical and numerical
categorical_columns = df.select_dtypes(include='object')
numerical_columns = df.select_dtypes(include=['int64','float64'])

In [44]:
#step 2 : Evaluate descriptive stats 
from collections import OrderedDict
import pandas as pd

# Assuming 'df' is your DataFrame
numerical_cols = df.select_dtypes(include=['int64', 'float64'])

stats = []
for i in numerical_cols.columns:
    numerical_stats = OrderedDict({
        'feature': i,
        'mean': numerical_cols[i].mean(),
        'median': df[i].median(),
        'mode': df[i].mode()[0],
        'min': df[i].min(),
        'max': df[i].max(),
        'std': df[i].std(),
        'var': df[i].var(),
        'skew': df[i].skew(),
        'kurtosis': df[i].kurtosis(),
        'range': df[i].max() - df[i].min(),
        'q1': df[i].quantile(0.25),
        'q3': df[i].quantile(0.75),
        'iqr': df[i].quantile(0.75) - df[i].quantile(0.25),
    })
    stats.append(numerical_stats)

report = pd.DataFrame(stats)


In [ ]:
#classification always msolve loggistic and loggistic based on sigmoid function
#varaince =  spread of the data
# if IQR is (0) thendrop the column
'''
whenever evaluate the descriptive statistics iQR cannot be zero
such column cannot be perform machine learning

'''